## de-Bruijn graph and Breadth First search in Genome Assembly

#### Intelligence of Biological Systems-3 project

In [1]:
def read_file():
    file = open('data1/short_1_dummy.fasta', 'r')
    reads = []
    lines = file.readlines()
    for line in lines:
        if not line.startswith(">"):
            reads.append(line.strip())
    file.close()
    return reads


In [2]:
def read_to_kmers(reads, k): #Converts reads to k-mers
    kmers_list = {} #Initialising a dictionary
    for read in reads:
        for i in range(len(read)-k+1):
            kmer = read[i:i+k] #string from i to i+k are taken as kmer
            if kmer not in kmers_list.keys():
                kmers_list[kmer] = 1
            else:
                kmers_list[kmer] += 1
    return kmers_list

In [11]:
def get_debruijn_edges_from_kmers(kmers):
    # store edges as tuples in a set
    edges = set()

    # compare each (k-1)mer
    for k1 in kmers:
        for k2 in kmers:
            if k1 != k2:
                # if they overlap then add to edges
                if k1[1:] == k2[:-1]:
                    edges.add((k1[:-1], k2[:-1]))
                if k1[:-1] == k2[1:]:
                    edges.add((k2[:-1], k1[:-1]))

    return edges


In [4]:
import toyplot

def plot_debruijn_graph(edges, width=700, height=700):
    "returns a toyplot graph from an input of edges"
    graph = toyplot.graph(
        [i[0] for i in edges],
        [i[1] for i in edges],
        width=width,
        height=height,
        tmarker=">",
        vsize=25,
        vstyle={"stroke": "black", "stroke-width": 2, "fill": "white"},
        vlstyle={"font-size": "11px"},
        estyle={"stroke": "black", "stroke-width": 2},
        layout=toyplot.layout.FruchtermanReingold(edges=toyplot.layout.CurvedEdges()))
    return graph


In [5]:
def get_dbg_from_edges(edges):

    graph = {} # stores the graph in a dictionary
    for edge in edges:
        if edge[0] not in graph:
            graph[edge[0]] = [edge[1]]
        else:
            graph[edge[0]].append(edge[1])

    return (graph)


In [6]:
def bfs(graph, node):
    visited = []  # List for visited nodes.
    queue = [] 
    BFS_traversal = []

    visited.append(node)
    queue.append(node)

    while queue:          # Creating loop to visit each node
        m = queue.pop(0)
        BFS_traversal.append(m)

        for neighbour in graph[m]:

            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(neighbour)
    return " -> ".join(BFS_traversal)


In [10]:
k = int(input("Enter the value of k : "))
reads = read_file() #Reads the contents from the file
kmers = read_to_kmers(reads, k) #Converts reads to k-mers
edges = get_debruijn_edges_from_kmers(kmers)
plot_debruijn_graph(edges) #plots the graph
graph = get_dbg_from_edges(edges)


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="700.0px" viewBox="0 0 700.0 700.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tb0e796b0f85f4f29a12da445bc3fbf52"> AA AC AG AT CA CC CG CT GA GC GG GT TA TC TG TT

In [8]:
print(f"BFS traversal of the de Bruijn Graph made by {k}-mers is: ")
print(bfs(graph, list(graph.keys())[0]))

BFS traversal of the de Bruijn Graph made by 4-mers is: 
TAT -> ATA -> ATG -> ATT -> ATC -> TAC -> TAG -> TAA -> TGA -> TGT -> TGG -> TGC -> TTT -> TTA -> TTC -> TTG -> TCG -> TCA -> TCC -> TCT -> ACA -> ACT -> ACG -> ACC -> AGG -> AGC -> AGA -> AGT -> AAG -> AAC -> AAT -> AAA -> GAT -> GAC -> GAA -> GAG -> GTA -> GTT -> GTG -> GTC -> GGT -> GGG -> GGA -> GGC -> GCG -> GCA -> GCT -> GCC -> CGA -> CGC -> CGT -> CGG -> CAT -> CAG -> CAA -> CAC -> CCG -> CCT -> CCC -> CCA -> CTA -> CTC -> CTG -> CTT
